# Event-log creator

In [ ]:
# IMPORTS
import pandas as pd

In [36]:
# Set filepath for saving
file_path_general = 'put_your_own_filepath_here'

In [37]:
# Read data
file_path = 'filepath/syntetic_merged_data.csv'
df = pd.read_csv(file_path ,sep=',')
df.head()

,Auftragsnummer,Versand_ID,Verladestart,Verladeende,Versand_SOLL,Lagereinheit_ID,Packplatz,Packzeitpunkt,Menge,Lagerort_von,Lagerort_nach,Startzeitpunkt,Endzeitpunkt
0,1002,33,26.11.2022 06:12,26.11.2022 08:29,26.11.2022 06:42,396,Packplatz 03,26.11.2022 05:26,458,Wareneingang,Hochregallager,2022-11-25 09:21:00,2022-11-25 10:05:00
1,1002,33,26.11.2022 06:12,26.11.2022 08:29,26.11.2022 06:42,396,Packplatz 03,26.11.2022 05:26,458,Hochregallager,Packplatz,2022-11-25 20:58:00,2022-11-25 21:38:00
2,1009,72,26.11.2022 05:00,26.11.2022 05:50,26.11.2022 05:30,492,Packplatz 07,26.11.2022 03:05,402,Hochregallager,Packplatz,2022-11-25 20:54:00,2022-11-25 23:08:00
3,1009,72,26.11.2022 05:00,26.11.2022 05:50,26.11.2022 05:30,492,Packplatz 07,26.11.2022 03:05,402,Wareneingang,Hochregallager,2022-11-25 22:13:00,2022-11-26 01:42:00
4,1030,16,26.11.2022 01:52,26.11.2022 02:04,26.11.2022 02:22,859,Packplatz 09,25.11.2022 22:22,61,Hochregallager,Packplatz,2022-11-25 18:33:00,2022-11-25 20:42:00


In [38]:
# Set process activities
process_activities = ['Ware vereinnahmen', 'Waren lagern', 'Waren kommissionieren']

# Set storage location
storage_locations = ['Wareneingang', 'Hochregallager', 'Packplatz']

In [39]:
# Make a list with all Auftragsnummern 
auftragsnummern = df['Auftragsnummer'].unique().tolist()

# Group data after Auftragsnummer
grouped_data = df.groupby('Auftragsnummer') #['Auftragsnummer', 'Lagereinheit_ID', 'Versand_ID']

# Make a new pandas dataframe
df_new = pd.DataFrame(columns=['case_id', 'process_activity', 'Auftragsnummer', 'Versand_ID', 'Verladestart', 'Verladeende', 'Versand_SOLL', 'Lagereinheit_ID', 'Packplatz', 'Packzeitpunkt', 'Menge', 'Lagerort_von', 'Lagerort_nach', 'Startzeitpunkt', 'Endzeitpunkt', 'Verspaetung', 'Lagerort'])

In [40]:
%%capture
# Get the WMS-movement rows for a unique event
# A unique event is defined by a 'Auftragsnummer', 'Lagereinheiten_ID' and 'Versand_ID. The combination of these figures also creates the case_id.
# The case_id shows all related WMS-movements.

# For the above descripted reason, there are three nested for-loops. Every loop groupes the dataframe further.
# The first input dataframe is a df grouped by 'Auftragsnummer', the second is grouped by 'Lagereinheit_ID' and the third by 'Versand_ID'.
# Lastly we get every time two WMS-movements and creates due to this the needed rows for the event-log. 

for i in auftragsnummern:
    case_data1 = grouped_data.get_group(i)
    lagereinheit_ids = case_data1['Lagereinheit_ID'].unique().tolist()
    grouped_lagereinheit_id = case_data1.groupby('Lagereinheit_ID')
    for j in lagereinheit_ids:
        case_data2 = grouped_lagereinheit_id.get_group(j)
        versand_ids = case_data2['Versand_ID'].unique().tolist()
        grouped_versand_id = case_data2.groupby('Versand_ID')
      
        for b in versand_ids:
            case_data = grouped_versand_id.get_group(b).sort_values(by='Lagerort_von', ascending=False).reset_index(drop=True)

            # Create case_id
            case_id = str(i) + '_' + str(j) + '_' + str(b)

            # Get current Index 
            current_index = case_data.index[0] 

            # Check if the process have a delay
            if case_data.at[current_index, 'Versand_SOLL'] < case_data.at[current_index, 'Verladeende']:
              delay = True
            else:
              delay = False
            
            # Choose current row
            current_row = case_data.iloc[0] 
            print(current_row)
            
            current_row_copy = current_row.copy()

            # Get values from current row
            cu_lagerort_nach = current_row['Lagerort_nach']
            cu_endzeitpunkt = current_row['Endzeitpunkt']
            cu_startzeitpunkt = current_row['Startzeitpunkt']

            # Get values from the directly following row
            next_row = case_data.iloc[1]
            ne_startzeitpunkt = next_row['Startzeitpunkt']
            ne_endzeitpunkt = next_row['Endzeitpunkt']

            # Prepare new line to implement
            current_row_copy['Lagerort_von'] = cu_lagerort_nach
            current_row_copy['Startzeitpunkt'] = cu_endzeitpunkt
            current_row_copy['Endzeitpunkt'] = ne_startzeitpunkt
            current_row_copy['case_id'] = case_id
            current_row_copy['process_activity'] = process_activities[1]
            current_row_copy['Verspaetung'] = delay
            current_row_copy['Lagerort'] = storage_locations[1]

            # Prepare lines
            # Modify current line
            case_data.at[current_index, 'Endzeitpunkt'] = cu_startzeitpunkt
            case_data.at[current_index,
                         'Startzeitpunkt'] = '??? '  
            case_data.at[current_index, 'case_id'] = case_id
            case_data.at[current_index, 'process_activity'] = process_activities[0]
            case_data.at[current_index, 'Verspaetung'] = delay
            case_data.at[current_index, 'Lagerort'] = storage_locations[0]

            # Modify the next following line
            case_data.at[current_index+1,
                         'Startzeitpunkt'] = ne_endzeitpunkt
            case_data.at[current_index+1,
                         'Endzeitpunkt'] = '???'  
            case_data.at[current_index+1, 'case_id'] = case_id
            case_data.at[current_index+1, 'process_activity'] = process_activities[2]
            case_data.at[current_index+1, 'Verspaetung'] = delay
            case_data.at[current_index+1, 'Lagerort'] = storage_locations[2]

            # Paste the copied line below the current line
            df2 = pd.concat([case_data.iloc[:current_index+1], current_row_copy.to_frame(
            ).T, case_data.iloc[current_index+1:]]).reset_index(drop=True)

            df_new = df_new.append(df2, ignore_index=True)
    
file_path = file_path_general + 'event-log.csv'
df_new.to_csv(file_path, index=False)
    

In [41]:
# Short event-log
# Creates rows only with the needed informations
df_short = df_new.copy()
df_short.drop(['Versand_ID', 'Verladestart', 'Verladeende', 'Versand_SOLL', 'Lagereinheit_ID', 'Packplatz', 'Packzeitpunkt'], axis=1, inplace=True)

file_path = file_path_general + "event-log_short.csv"
df_short.to_csv(file_path, index=False)